In [1]:
import pandas as pd
import utils
import sklearn
#import gensim

In [13]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.parsing.preprocessing import stem_text
#from utils import word_frequency_per_player
def word_frequency_per_player(df, playerlist):
    '''
    Function which returns the frequncy of words in articles for all players 
    '''
     # define empty df which will be returned in the end
    df_complete = pd.DataFrame()

    for player in playerlist:

         # create the df for the player
        df_player = df[df["player"] == player]
        df_player = df_player.dropna(subset=['data'])
        df_player = df_player.reset_index(drop=True)

        # create a stemmed data corpus
        df_player['stemmed_data'] = df_player['data'].apply(stem_text)
        data_stem = df_player['data'].apply(stem_text)
        data = data_stem.tolist()

        # create a corpus
        corpus_gen=[doc.split() for doc in data]

        # Assume `corpus` is a preprocessed corpus
        id2word = Dictionary(corpus_gen)

        # Filter out rare and common words
        id2word.filter_extremes(no_below=5, no_above=0.95)

        # Display features and their frequencies
        df_frequencies = pd.DataFrame(columns=['Word', 'Frequency', 'player'])
    
        i = 1
        for feature, frequency in id2word.cfs.items():

            # Append a new row to the DataFrame
            df_frequencies.loc[i]= [id2word[feature],frequency, player]
            i = i+1

        df_frequencies = df_frequencies.sort_values('Frequency', ascending=False)

        df_complete = pd.concat([df_complete, df_frequencies], axis=0)

    return df_complete


# Load Data

In [2]:
#dataset 2 is for text clustering
url_en2 = 'https://github.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/raw/main/Preprocessing/data_clean/en_clean_2.csv'
df_en2 = pd.read_csv(url_en2, sep=',', encoding='utf-8', error_bad_lines=False)

C:\Users\joana\AppData\Local\Temp\ipykernel_2980\2467272692.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_en2 = pd.read_csv(url_en2, sep=',', encoding='utf-8', error_bad_lines=False)


In [3]:
df_en2

,data,player,language,publishedAt
0,footballflorian wirtz goal bayer leverkusen eu...,exequiel palacios,en,2023-02-16T23:56:00Z
1,xasoccer football europa league play second le...,exequiel palacios,en,2023-02-23T20:50:50Z
2,pickworth mailonlineview commentsbayer leverku...,exequiel palacios,en,2023-02-23T20:53:59Z
3,buenos aires world cup winner argentina celebr...,exequiel palacios,en,2023-03-03T16:40:46Z
4,sign insign inthe star editionchange locationt...,exequiel palacios,en,2023-03-03T16:42:19Z
...,...,...,...,...
284,australian associated pressview commentsthe so...,piero hincapie,en,2023-03-28T11:24:40Z
285,julien laurens explains choice espn top star w...,piero hincapie,en,2023-03-29T11:35:08Z
286,"futurechelsea rumored beaten ""up dozen"" teams ...",piero hincapié,en,2023-04-27T04:57:02Z
287,man city's alex robertson makes debut aiden o'...,piero hincapié,en,2023-03-24T15:24:08Z


# Trying sklearn tfidfVectorizer

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
'''info: by default this doesn't use a stopword list, but it does remove accents and transforms the data into lowercase'''
from sklearn.compose import ColumnTransformer

In [5]:
vec_count = CountVectorizer(max_df=0.95, min_df=0.05)
vec = TfidfVectorizer(max_df=0.95, min_df=0.05) #smoothing and inverse document frequency (= use_idf) enabled, to prevent zero devisions and have the real tfidf not only term frequency
vec_term = TfidfVectorizer(max_df=0.95, min_df=0.05,use_idf=False, norm='l1') #only term frequency is disabled

#it should only be performed on the column data, so we use ColumnTransformer to uapply tfidf selectively
columns_to_remain = ['player', 'language', 'publishedat']
column_transformer = ColumnTransformer(
    transformers=[
        ('tfidf', vec, df_en2['data']),
        ('remainder', 'passthrough', columns_to_remain)  # Include the remaining columns as-is
    ])

In [7]:
###this is where I stopped before
###transformed_data = column_transformer.fit_transform(df_en2)

In [8]:
###not working yet, the way I wanted
#tfidf_count = vec_count.fit_transform(df_en2).toarray()
#tfidf = vec.fit_transform(df_en2).toarray()
#tfidf_term = vec_term.fit_transform(df_en2).toarray()

In [9]:
###not working yet, the way I wanted
#print('countvectorizer', tfidf_count)
#print('tfidf:', tfidf)
#print('term frequency:', tfidf_term)

countvectorizer [[1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 0 1]]
tfidf: [[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
term frequency: [[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [10]:
###not working yet, the way I wanted
#df_tfidf = pd.DataFrame(tfidf, columns=vec.vocabulary_.keys())
#df_tfidf.head()

,data,player,language,publishedat
0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0


In [15]:
df_stem = df_en2.copy () 
df_stem = df_stem.dropna(subset=['data'])
# create column with stemmed data
df_stem['stemmed_data'] = df_stem['data'].apply(stem_text) #this is still gensim

# create a corpus
data_stem = df_stem['stemmed_data'].apply(stem_text)
data = data_stem.tolist()

# create and fit vectorizer
vectorizerTfid = TfidfVectorizer(max_df=0.95, min_df=0.05, smooth_idf=False)
dataTfid = vectorizerTfid.fit_transform(data)

# Display features and their frequencies
df_frequencies = pd.DataFrame(columns=['Word', 'Frequency'])
i = 1
for feature, frequency in vectorizerTfid.vocabulary_.items():
    # Append a new row to the DataFrame
    df_frequencies.loc[i] = [feature, frequency]
    i += 1
df_frequencies = df_frequencies.sort_values('Frequency', ascending=False)

df_freq_en = word_frequency_per_player(df_en2, df_en2['player'].unique())

# Subset for rows in Categoric
def print_player_freq(df_freq):
    for player in df_freq['player'].unique():
        # Delete rows we don't want to see e.g. names etc.
        # Define the strings to be deleted
        strings_to_delete = ['palacio', 'leverkusen', 'bayer', 'exequiel', 'frimpong', 'jeremi', 'tah', 'jonathan', 'bakker', 'mitchel', 'fur', 'moussa', 'diaby', 'mykhaylo', 'piero', 'hincapie', 'nicht', 'diabi']

        # Delete rows containing the specified strings 
        df_freq = df_freq[~df_freq['Word'].isin(strings_to_delete)]

        df_player = df_freq[(df_freq["player"] == player)]
        print('Top Words for', player, '\n', df_player.head(20), '\n')

# print frequencies per player for english data
print_player_freq(df_freq_en)


Top Words for exequiel palacios 
            Word Frequency             player
1          goal       218  exequiel palacios
69       bayern       195  exequiel palacios
38          win       172  exequiel palacios
54        point       156  exequiel palacios
30        minut       155  exequiel palacios
20        score       153  exequiel palacios
162       union       152  exequiel palacios
5         leagu       124  exequiel palacios
142        lead       112  exequiel palacios
19         time       112  exequiel palacios
198   frankfurt       106  exequiel palacios
95       second       105  exequiel palacios
189    dortmund        95  exequiel palacios
400     khedira        94  exequiel palacios
18         ball        90  exequiel palacios
133        home        88  exequiel palacios
63   bundesliga        86  exequiel palacios
96         half        81  exequiel palacios
122     penalti        77  exequiel palacios
91         awai        73  exequiel palacios 

Top Words for jerem

# Trying gensims tfidfModel

In [14]:
df_stem = df_en2.copy () #QUESTION: can we call this df_stem? Is it really stemmed?
df_stem = df_stem.dropna(subset=['data'])
# create column with stemmed data
df_stem['stemmed_data'] = df_stem['data'].apply(stem_text)

# create a corpus
data_stem = df_stem['stemmed_data'].apply(stem_text)
data = data_stem.tolist()

# create a corpus
corpus_gen = [doc.split() for doc in data]
id2word = Dictionary(corpus_gen)

# Filter out rare and common words
id2word.filter_extremes(no_below=2, no_above=0.99)

# create and fit vectorizer
corpus = [id2word.doc2bow(doc) for doc in corpus_gen]
tfidf_model = TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]

# Display features and their frequencies
df_frequencies = pd.DataFrame(columns=['Word', 'Frequency'])
i = 1
for word_id, frequency in id2word.cfs.items():
    # Append a new row to the DataFrame
    df_frequencies.loc[i] = [id2word[word_id], frequency]
    i += 1
df_frequencies = df_frequencies.sort_values('Frequency', ascending=False)

df_freq_en = word_frequency_per_player(df_en2, df_en2['player'].unique())

# Subset for rows in Categoric
def print_player_freq(df_freq):
    for player in df_freq['player'].unique():
        # Delete rows we don't want to see e.g. names etc.
        # Define the strings to be deleted
        strings_to_delete = ['palacio', 'leverkusen', 'bayer', 'exequiel', 'frimpong', 'jeremi', 'tah', 'jonathan', 'bakker', 'mitchel', 'fur', 'moussa', 'diaby', 'mykhaylo', 'piero', 'hincapie', 'nicht', 'diabi']

        # Delete rows containing the specified strings 
        df_freq = df_freq[~df_freq['Word'].isin(strings_to_delete)]

        df_player = df_freq[(df_freq["player"] == player)]
        print('Top Words for', player, '\n', df_player.head(20), '\n')

# print frequencies per player for english data
print_player_freq(df_freq_en)

Top Words for exequiel palacios 
            Word Frequency             player
1          goal       218  exequiel palacios
69       bayern       195  exequiel palacios
38          win       172  exequiel palacios
54        point       156  exequiel palacios
30        minut       155  exequiel palacios
20        score       153  exequiel palacios
162       union       152  exequiel palacios
5         leagu       124  exequiel palacios
142        lead       112  exequiel palacios
19         time       112  exequiel palacios
198   frankfurt       106  exequiel palacios
95       second       105  exequiel palacios
189    dortmund        95  exequiel palacios
400     khedira        94  exequiel palacios
18         ball        90  exequiel palacios
133        home        88  exequiel palacios
63   bundesliga        86  exequiel palacios
96         half        81  exequiel palacios
122     penalti        77  exequiel palacios
91         awai        73  exequiel palacios 

Top Words for jerem

# Comparing the performance of sklearn and gensim

In [16]:
import time

In [17]:
# Using sklearn
start_time_sklearn = time.time()
vectorizer = TfidfVectorizer()
tfidf_matrix_sklearn = vectorizer.fit_transform(df_stem)
end_time_sklearn = time.time()

# Using gensim
start_time_gensim = time.time()
corpus_gen = [doc.split() for doc in df_stem]
id2word = Dictionary(corpus_gen)
corpus = [id2word.doc2bow(doc) for doc in corpus_gen]
tfidf_model = TfidfModel(corpus)
corpus_tfidf_gensim = tfidf_model[corpus]
end_time_gensim = time.time()

# Calculate execution time
execution_time_sklearn = end_time_sklearn - start_time_sklearn
execution_time_gensim = end_time_gensim - start_time_gensim

# Print execution time
print("Execution time using sklearn: {:.6f} seconds".format(execution_time_sklearn))
print("Execution time using gensim: {:.6f} seconds".format(execution_time_gensim))


Execution time using sklearn: 0.016145 seconds
Execution time using gensim: 0.048175 seconds


We cshould use sklearn, because it's faster and the results of the two modeling packages should and are the same (the core TF-IDF calculation remains the same across different implementations.).